In [1]:
%matplotlib inline  
import mxnet as mx
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt

from vision.cnn import resnet

In [2]:
data_dir = '/media/tx-deepocean/Data/dataset/cifar10/cifar-10-batches-py'

train_files = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
test_file = 'test_batch'

In [3]:
data = []
label = []

for train_file in train_files:
    with open(os.path.join(data_dir, train_file), 'rb') as fo:
        tmp = pickle.load(fo, encoding='bytes')
        data.append(np.reshape(tmp[b'data'], (tmp[b'data'].shape[0], 3, 32, 32)))
        label.extend(tmp[b'labels'])
data = np.concatenate(data, axis=0)

In [4]:
test_data = []
test_label = []

with open(os.path.join(data_dir, test_file), 'rb') as fo:
        tmp = pickle.load(fo, encoding='bytes')
        test_data = np.reshape(tmp[b'data'], (tmp[b'data'].shape[0], 3, 32, 32))
        test_label = tmp[b'labels']

In [32]:
batch_size = 500
# oh_label = mx.nd.one_hot(mx.nd.array(label), 10)
train_iter = mx.io.NDArrayIter(data, mx.nd.array(label), batch_size=batch_size, shuffle=True)
oh_label = mx.nd.one_hot(mx.nd.array(test_label), 10)
val_iter = mx.io.NDArrayIter(test_data,mx.nd.array(test_label), batch_size=batch_size)

In [18]:
eval_acc = mx.metric.Accuracy()
eval_loss = mx.metric.CrossEntropy(label_names=['softmax_label'])

eval_metrics = mx.metric.CompositeEvalMetric()

for child_metric in [eval_acc, eval_loss]:
    eval_metrics.add(child_metric)

In [19]:
r18 = resnet.resnet18(classes=10)
r18_model = mx.mod.Module(symbol=r18.output, context=[mx.gpu(0),mx.gpu(1)])

In [20]:
import logging
logging.getLogger().setLevel(logging.DEBUG)
r18_model.fit(train_data=train_iter,
              eval_data=val_iter,
              optimizer='sgd', 
              optimizer_params={'learning_rate': 0.001},
              eval_metric=eval_metrics,
              num_epoch=30)

INFO:root:Epoch[0] Train-accuracy=0.235576
INFO:root:Epoch[0] Train-cross-entropy=2.151266
INFO:root:Epoch[0] Time cost=4.030
INFO:root:Epoch[0] Validation-accuracy=0.320800
INFO:root:Epoch[0] Validation-cross-entropy=1.929251
INFO:root:Epoch[1] Train-accuracy=0.351480
INFO:root:Epoch[1] Train-cross-entropy=1.820826
INFO:root:Epoch[1] Time cost=3.979
INFO:root:Epoch[1] Validation-accuracy=0.388400
INFO:root:Epoch[1] Validation-cross-entropy=1.718810
INFO:root:Epoch[2] Train-accuracy=0.408940
INFO:root:Epoch[2] Train-cross-entropy=1.659131
INFO:root:Epoch[2] Time cost=3.977
INFO:root:Epoch[2] Validation-accuracy=0.426700
INFO:root:Epoch[2] Validation-cross-entropy=1.608232
INFO:root:Epoch[3] Train-accuracy=0.448280
INFO:root:Epoch[3] Train-cross-entropy=1.553018
INFO:root:Epoch[3] Time cost=3.988
INFO:root:Epoch[3] Validation-accuracy=0.455500
INFO:root:Epoch[3] Validation-cross-entropy=1.532577
INFO:root:Epoch[4] Train-accuracy=0.476940
INFO:root:Epoch[4] Train-cross-entropy=1.477202
I